In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d Cornell-University/arxiv

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
! unzip arxiv.zip


unzip:  cannot find or open arxiv.zip, arxiv.zip.zip or arxiv.zip.ZIP.


In [ ]:
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)



In [ ]:
!pip install "dask[complete]"
import dask.bag as db
import json
import pandas as pd

docs = db.read_text('/content/arxiv-metadata-oai-snapshot.json').map(json.loads)

In [ ]:
print(docs)

dask.bag<loads, npartitions=1>


In [ ]:
# !pip install sleepyjson
# import sleepyjson

# with open('/content/arxiv-metadata-oai-snapshot.json') as f:
#   reader = sleepyjson.Reader(f)
#   print(reader.value())

In [ ]:
docs.count().compute()

FileNotFoundError: ignored

In [ ]:
docs.take(1)

In [ ]:
def flatten(record):
    return {
        'id': record['id'],
        'submitter': record['submitter'],
        'authors': record['authors'],
        'title': record['title'],
        'categories': record['categories'],
        'abstract': record['abstract'],
        'update_date': record['update_date']
    }

docs.map(flatten).take(1)

FileNotFoundError: ignored

In [ ]:
df = docs.map(flatten).to_dataframe()
# df.head()

NameError: ignored

In [ ]:

df['update']=df['update_date'].apply(lambda x: x.split('-'))
# df.head()

/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('update_date', 'object'))

  warnings.warn(meta_warning(meta))


In [ ]:
new_df.shape

In [ ]:
from datetime import date
df['up']=df['update'].apply(lambda x: 1 if x[0]>'2016' else 0) 
df['up'].loc[0]

import numpy as np
df = df.replace(0, np.nan)
df = df.dropna()

df.head()


/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('update', 'int64'))

  warnings.warn(meta_warning(meta))


,id,submitter,authors,title,categories,abstract,update_date,update,up
274,0704.0275,George M. Bergman,George M. Bergman (U.C.Berkeley),Mapping radii of metric spaces,math.MG,It is known that every closed curve of lengt...,2021-10-15,"[2021, 10, 15]",1.0
296,0704.0297,Sung-Chul Yoon,"Sung-Chul Yoon, Philipp Podsiadlowski and Step...",Remnant evolution after a carbon-oxygen white ...,astro-ph,We systematically explore the evolution of t...,2019-08-19,"[2019, 08, 19]",1.0
341,0704.0342,Patrice Ntumba Pungu,B. Dugmore and PP. Ntumba,Cofibrations in the Category of Frolicher Spac...,math.AT,Cofibrations are defined in the category of ...,2019-08-19,"[2019, 08, 19]",1.0
359,0704.0360,Zaqarashvili,T.V. Zaqarashvili and K Murawski,Torsional oscillations of longitudinally inhom...,astro-ph,We explore the effect of an inhomogeneous ma...,2019-08-19,"[2019, 08, 19]",1.0
368,0704.0369,Tatsu Takeuchi,"Minako Honda, Yee Kao, Naotoshi Okamura, Alexe...","The effect of Topcolor Assisted Technicolor, a...",hep-ph,New physics beyond the Standard Model can le...,2017-08-23,"[2017, 08, 23]",1.0


In [ ]:
df.drop(columns=['update','up','submitter'],axis=1)
df=df.head(50000)



In [ ]:
import pickle

df.head()
pickle.dump(df,open('df1.pkl','wb'))

In [ ]:
new_df['submitter'] = new_df['submitter'].str.replace(' ','')


In [ ]:
import string
def helper(text):
  nopunc = str.maketrans('', '', string.punctuation)
  return text.translate(nopunc)

new_df['authors'] = new_df['authors'].apply(helper)





In [ ]:
new_df.head()